# クオンツ選考課題

## するべきこと
- 期待リスク、リターン、シャープレシオを計算する
- リターンを一定に保つ。(今回は期待リスクとする)。目標値は4%
- 運用結果は図で示す。

## 変数とその働き
- table: 各銘柄の評価指標を最新時点のデータで蓄積
- data: yfinanceから取得した10年分の価格データ

## 前提や仕様
- 日経225をポートフォリオの候補とする。
- Closeを使用する。株式分割や配当による影響を除いた修正後の終値を用いることで計算に整合性が取れるため。
- ポートフォリオは225銘柄のweightで表現される。
- 各銘柄について、累計の期待リスク、リターン、シャープレシオを用意しておく。
- 10年の過去データでバックテストを行う。
- 3年分は基準指標の計算に利用し、7年分ポートフォリオ運用を回し、結果を分析する。
- 実際の運用を想定し、7年分のデータは1日ずつ流れ込んでくるとする(毎日このコードを回して更新する設定)。
- 実際の運用において通用する関数を設定するべき。つまり、バックテスト用の設計に絞る必要はない。
- 無リスク金利はhttps://www.boj.or.jp/statistics/dl/loan/prime/prime.htmlに基づき、1.8%と仮に定める。
- 売買手数料は考慮しない。
- 借入レートと貸出レートに違いはないとする。

## 想定する処理の流れ
1. データを10年分読み込む。
2. ポートフォリオ調整を行う関数を用意する。
3. 運用指標を計算する関数を設定。 
4. 3年分の運用指標を計算。
5. 7年分のデータを1日ずつ流し込み、データを蓄積する。
6. 運用実績の推移を図示めする。

## 気になること
- 実際の運用において、更新のタイミングやデータ処理スピードの要求値はどれくらいか

## AIを仕様した箇所
- yfinanceのセットアップと銘柄選定
日経225の銘柄のデータをyfinanceから取得するまでのプロセスにおいて、AIの助けを借りました。
- 最適化問題の実装において
最適なウェイトを見つける上でPythonで実装する方法がわからなかったため、助けを借りました。

In [108]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [109]:
# 日経225のデータを取得

# https://indexes.nikkei.co.jp/nkave/index/component?idx=nk225 から自身で作成 
# table = pd.read_csv("日経225.csv",header=None)
# table.columns=["コード","会社名"]
# table["コード"] = table["コード"].astype(str) + ".T"
# data = yf.download([code for code in table["コード"]], period="10y")
# closeData = data["Close"] これが銘柄のデータ全体
# pctData = data["Close"].pct_change() これが%での変化
# returnData = (1+pctData).cumprod() これがリターン

In [110]:
testdata = yf.download(["1802.T","1721.T","1928.T"], period="3y")
returnData = testdata["Close"].pct_change().dropna()

returnData.loc["2023":"2025"]


[*********************100%***********************]  3 of 3 completed


Ticker,1721.T,1802.T,1928.T
Date,,,
2023-02-21,0.003620,0.009202,0.002357
2023-02-22,-0.014028,-0.003040,-0.008426
2023-02-24,0.012602,0.002033,0.005336
2023-02-27,0.006423,0.032454,0.014154
2023-02-28,-0.011568,-0.010806,0.000000
...,...,...,...
2025-12-24,0.000000,0.004281,0.000875
2025-12-25,0.015061,0.005786,0.014277
2025-12-26,-0.001309,-0.006661,-0.002011


In [111]:
testdata = yf.download(["1802.T","1721.T","1928.T"], period="10y")
returnData = testdata["Close"].pct_change().dropna()

print(type(returnData.index))
print(returnData.index[:5])
print(returnData.columns)

[*********************100%***********************]  3 of 3 completed

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['2016-02-19', '2016-02-22', '2016-02-23', '2016-02-24',
               '2016-02-25'],
              dtype='datetime64[ns]', name='Date', freq=None)
Index(['1721.T', '1802.T', '1928.T'], dtype='object', name='Ticker')


In [112]:
expectedReturn = returnData.aggregate(np.mean) #期待リターン
expectedVariance = returnData.cov() #共分散行列


/var/folders/_5/80rp92qj06xby1j6gc1k5r100000gn/T/ipykernel_99779/3180585650.py:1: FutureWarning: The provided callable <function mean at 0x10c817a60> is currently using DataFrame.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  expectedReturn = returnData.aggregate(np.mean) #期待リターン


# AIへの指示
すでにある価格データからリターン(returnData)が作成できる
そこから期待リターンは
expectedReturn = returnData.aggregate(np.mean)
でそれぞれの銘柄について把握できて、全体の共分散行列は
expectedVariance = returnData.cov()
によって計算できる。ここまではPythonでOK

ここにおける最適なweightは、目標リターンを毎年
数学的に言えば、$\bar{R}_i$をその銘柄の期待リターンとして
$$\min_{w \in R^n} \{Var(\sum_{i=1}^{n} w_i \bar{R}_i)\}$$
$$s.t. \sum_{i=1}^{n} w_i \bar{R}_= 0.04$$
を生み出す$\{w_i\}_i$がその時点の最適ポートフォリオになる。 

この$w_i$のリストを吐き出す関数を作ってください

In [113]:
def findInitialValue (rtrData:pd.DataFrame,target_return:int):
    """
    3年分のデータを踏まえて初期時点でのウェイトを計算する。各銘柄の期待リターンと共分散行列から、目標リターンを実現する最小分散のウェイトを計算しspilts outする。
    rtrData: n銘柄のリターンのデータ
    target_return : 目標のリターン
    """
    expectedReturn = rtrData.aggregate(np.mean)
    expectedVariance = rtrData.cov()

    mu = expectedReturn.values
    Sigma = expectedVariance.values
    n_assets = len(mu)

    # 目的関数: ポートフォリオ分散 w^T Σ w
    def portfolio_variance(w, Sigma):
        return w @ Sigma @ w

    # 制約1: 期待リターンが target_return
    # Σ w_i * μ_i = target_return
    def constraint_return(w, mu, target):
        return w @ mu - target

    # 制約2: 重みの合計が 1 （フルインベスト）
    def constraint_weight_sum(w):
        return np.sum(w) - 1.0

    # 初期値: 等ウェイト
    w0 = np.ones(n_assets) / n_assets

    # 制約設定 (type = "eq" は「= 0」という意味)
    constraints = [
        {
            "type": "eq",
            "fun": lambda w, mu=mu, target=target_return: constraint_return(w, mu, target)
        },
        {
            "type": "eq",
            "fun": lambda w: constraint_weight_sum(w)
        }
    ]
    # 最適化実行
    result = minimize(
        fun=portfolio_variance,
        x0=w0,
        args=(Sigma,),
        method="SLSQP",
        constraints=constraints,
    )

    if not result.success:
        raise RuntimeError("最適化に失敗しました: " + result.message)

    w_opt = result.x
    return_opt = w_opt @ mu
    variance_opt = w_opt @ Sigma @ w_opt
    return w_opt,return_opt,variance_opt

In [114]:
findInitialValue (returnData,0.04)

/var/folders/_5/80rp92qj06xby1j6gc1k5r100000gn/T/ipykernel_99779/1660114919.py:7: FutureWarning: The provided callable <function mean at 0x10c817a60> is currently using DataFrame.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  expectedReturn = rtrData.aggregate(np.mean)


(array([  18.64497406,  128.88402699, -146.52900106]),
 0.04000000102285845,
 4.266516448483525)

In [118]:
begin = returnData.iloc[0].name.year
returnData.loc[f"{begin+3}":].index


DatetimeIndex(['2019-01-04', '2019-01-07', '2019-01-08', '2019-01-09',
               '2019-01-10', '2019-01-11', '2019-01-15', '2019-01-16',
               '2019-01-17', '2019-01-18',
               ...
               '2026-02-04', '2026-02-05', '2026-02-06', '2026-02-09',
               '2026-02-10', '2026-02-12', '2026-02-13', '2026-02-16',
               '2026-02-17', '2026-02-18'],
              dtype='datetime64[ns]', name='Date', length=1737, freq=None)

In [ ]:
def runBackTest(rtrData:pd.DataFrame,target_return):
    """
    バックテストを回す。先に3年分のデータを用いて初期のウェイトを計算し、その後そのポートフォリオの期待リターンをtarget_returnにして推移を確認する。
    ウェイトの推移を変数として、ポートフォリオの期待リターン、期待リスク、シャープレシオをグラフで表示する。
    returnData: リターンの10年分のデータ
    target_return: 初めのポートフォリオウェイト選定のための日次目標リターン
    """
    beginingYear = rtrData.iloc[0].name.year
    explorationData = rtrData.loc[f"{beginingYear}":f"{beginingYear+2}"]
    backtestDates = rtrData.loc[f"{begin+3}":].index
    n_dates = len(backtestDates)

    weight_history = pd.Series(index=range(0, n_dates+1), dtype=float)
    return_history = pd.Series(index=range(0, n_dates+1), dtype=float)
    Var_history = pd.Series(index=range(0, n_dates+1), dtype=float)

    initialWeight,initialReturn, initialVar = findInitialValue (explorationData,target_return)
    weight_history[0] = initialWeight
    return_history[0] = initialReturn
    Var_history[0] = initialVar

    for i,d in enumerate(backtestDates):
        testdata = rtrData.iloc[:d]
        nextWeight,nextReturn, nextVar = findInitialValue (testdata,initialReturn)
        weight_history[i+1] = nextWeight
        return_history[i+1] = nextReturn
        Var_history[i+1] = nextVar

    

    
    

SyntaxError: invalid syntax (1735918432.py, line 22)

In [ ]:
def improveWeight(rtrData:pd.DataFrame,target_return:int) -> :
    """
    更新された銘柄の価格データを取得し、最新時点の最適ポートフォリオのウェイトを計算する
    rtrData: 更新済みのリターンデータ
    target_return: 初期のウェイトにおけるリターンを設置
    """
    w_next, return_next, var_next = findInitialValue(rtrData,target_return)
    

SyntaxError: expected ':' (2770913734.py, line 1)